In [27]:
import gymnasium as gym
import numpy as np
import random
from tqdm import tqdm
from PLUMP_ENV import CardDeck, PlumpPlayer, PlumpEnv
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Example usage:
num_players = 4
table = PlumpEnv(num_players)
table.deal_cards(num_cards = 8)
table.get_estimates()

In [3]:
for i, p in enumerate(table.players):
    print(f"Player {i}")
    for card in p.hand:
        print(f"{card['rank']}: {card['suit']}")
    print('-.-'*10)

Player 0
6: Hearts
10: Diamonds
5: Clubs
8: Spades
3: Spades
8: Clubs
Ace: Clubs
3: Clubs
-.--.--.--.--.--.--.--.--.--.-
Player 1
6: Clubs
3: Diamonds
4: Spades
Ace: Spades
2: Clubs
8: Diamonds
King: Spades
10: Hearts
-.--.--.--.--.--.--.--.--.--.-
Player 2
4: Hearts
7: Spades
3: Hearts
Jack: Spades
Queen: Spades
King: Clubs
9: Clubs
6: Spades
-.--.--.--.--.--.--.--.--.--.-
Player 3
Queen: Diamonds
2: Diamonds
Jack: Clubs
5: Spades
4: Clubs
King: Diamonds
10: Spades
8: Hearts
-.--.--.--.--.--.--.--.--.--.-


In [4]:
table.play_the_hands()

In [23]:
num_players = 4  # Change the number of players as needed
stick_strategies =['Royal Count'] *4
assert len(stick_strategies)== num_players, f"You have {len(stick_strategies)} strategies but only {num_players}!"
env = PlumpEnv(num_players, stick_strategies)

num_hands = 10
if num_hands < 1 or num_hands > 20:
    print("Invalid input. Please enter a value between 20 and 1.")
    exit()

go_down_then_up = True  
if go_down_then_up:
    range_values = range(num_hands, 0, -1)  # Count down
    range_values = list(range_values) + list(range(1, num_hands + 1))  # Count up
else:
    range_values = range(num_hands, 0, -1)  # Count down

for num_cards in range_values:
    print(num_cards)
    env.play_full_hand(num_cards=num_cards)

# Print the final results and protocol
print("Final Results:")
for player in env.players:
    print(f"Player {player.player_id}: Points={player.points}")

print("Protocol:")
for player_id, points_list in env.protocol.items():
    print(f"Player {player_id}: {points_list}")

10
9
8
7
6
5
4
3
2
1
1
2
3
4
5
6
7
8
9
10
Final Results:
Player 0: Points=114
Player 1: Points=84
Player 2: Points=69
Player 3: Points=93
Protocol:
Player 0: [13, 0, 12, 11, 11, 0, 12, 0, 0, 0, 0, 11, 0, 10, 0, 11, 0, 0, 11, 12]
Player 1: [0, 0, 0, 11, 10, 11, 0, 10, 11, 10, 0, 10, 0, 0, 11, 0, 0, 0, 0, 0]
Player 2: [0, 13, 0, 0, 0, 0, 10, 0, 0, 0, 10, 0, 11, 0, 12, 0, 0, 0, 13, 0]
Player 3: [0, 0, 0, 0, 13, 12, 11, 0, 11, 11, 0, 11, 0, 0, 11, 13, 0, 0, 0, 0]


In [42]:
def run_simulation(num_players, num_hands, stick_strategies, go_down_then_up):
    assert len(stick_strategies) == num_players, f"You have {len(stick_strategies)} strategies but {num_players} players!"

    env = PlumpEnv(num_players, stick_strategies)
    if num_hands < 1 or num_hands > 20:
        print("Invalid input. Please enter a value between 20 and 1.")
        exit()

    if go_down_then_up:
        range_values = range(num_hands, 0, -1)  # Count down
        range_values = list(range_values) + list(range(1, num_hands + 1))  # Count up
    else:
        range_values = range(num_hands, 0, -1)  # Count down

    for num_cards in range_values:
        env.play_full_hand(num_cards=num_cards)

    return [player.points for player in env.players]

In [60]:
import numpy as np

num_players = 4  # Change the number of players as needed
num_simulations = 1000  # Number of Monte Carlo simulations
stick_strategies1 =['Royal Count'] *4
stick_strategies2 =['Random'] *4


# Run Monte Carlo simulations
go_down_then_up = True
results1 = {}
results2 = {}
for n in tqdm((range(10, 0, -1)), unit= 'Hand size'):
    result1 = []
    result2 = []
    for _ in range(num_simulations):
        points1 = run_simulation(num_players=num_players, num_hands=n, go_down_then_up=go_down_then_up, stick_strategies=stick_strategies1)
        points2 = run_simulation(num_players=num_players, num_hands=n, go_down_then_up=go_down_then_up, stick_strategies=stick_strategies2)
        result1.append(np.mean(points1))
        result2.append(np.mean(points2))
    average_points1 = np.mean(result1)
    average_points2 = np.mean(result2)
    results1[n] = average_points1
    results2[n] = average_points2

print(f"Strategy {stick_strategies1}")
for n, p in results1.items():
    print(f"Given starting num_hands = {n}, the expected value is = {p}")
print('-.-'*20)
print(f"Strategy {stick_strategies2}")
for n, p in results2.items():
    print(f"Given starting num_hands = {n}, the expected value is = {p}")


  0%|          | 0/10 [00:00<?, ?Hand size/s]

100%|██████████| 10/10 [00:31<00:00,  3.13s/Hand size]

Strategy ['Royal Count', 'Royal Count', 'Royal Count', 'Royal Count']
Given starting num_hands = 10, the expected value is = 76.056
Given starting num_hands = 9, the expected value is = 69.445
Given starting num_hands = 8, the expected value is = 63.214
Given starting num_hands = 7, the expected value is = 57.67125
Given starting num_hands = 6, the expected value is = 51.28825
Given starting num_hands = 5, the expected value is = 43.64625
Given starting num_hands = 4, the expected value is = 37.03825
Given starting num_hands = 3, the expected value is = 30.125
Given starting num_hands = 2, the expected value is = 22.769
Given starting num_hands = 1, the expected value is = 13.823
-.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.-
Strategy ['Random', 'Random', 'Random', 'Random']
Given starting num_hands = 10, the expected value is = 43.00525
Given starting num_hands = 9, the expected value is = 40.263
Given starting num_hands = 8, the expected value is = 38.3245
Given startin

In [63]:

num_simulations = 1000  # Number of Monte Carlo simulations
num_hands_range = range(10, 0, -1)
num_players_range = range(5, 1, -1)  # From 5 to 2 players


# Run Monte Carlo simulations for different combinations of num_hands and num_players
go_down_then_up = True
results1 = {}
results2 = {}
for num_hands in tqdm(num_hands_range, unit='Hands'):
    results1[num_hands] = {}
    results2[num_hands] = {}
    for num_players in tqdm(num_players_range, unit='Players', leave=False):
        for _ in range(num_simulations):
            stick_strategies1 =['Royal Count'] * num_players
            stick_strategies2 =['Random'] * num_players
            assert num_players*num_hands <= 52, f"Too many cards need to be dealt {num_hands*num_players} > 52"
            result1 = run_simulation(num_players, num_hands, stick_strategies1, go_down_then_up)
            result2 = run_simulation(num_players, num_hands, stick_strategies2, go_down_then_up)
            
            results1[num_hands][num_players] = np.mean(result1)
            results2[num_hands][num_players] = np.mean(result2)

100%|██████████| 10/10 [01:51<00:00, 11.16s/Hands]


In [64]:

for num_hands in num_hands_range:
    print(f"# of hands\t# of players\tRoyal Count-Strategy\tRandom-strategy")
    print('-.-'*25)
    for num_players in num_players_range:
        expected_value1 = results1[num_hands][num_players]
        expected_value2 = results2[num_hands][num_players]
        print(f"{num_hands}\t\t{num_players}\t\t{expected_value1:.3f}\t\t\t{expected_value2:.3f}")
    print()

# of hands	# of players	Royal Count-Strategy	Random-strategy
-.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.-
10		5		53.400			53.000
10		4		73.500			43.250
10		3		69.000			37.333
10		2		66.500			65.000

# of hands	# of players	Royal Count-Strategy	Random-strategy
-.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.-
9		5		78.000			44.400
9		4		44.250			45.500
9		3		60.333			45.333
9		2		53.000			58.000

# of hands	# of players	Royal Count-Strategy	Random-strategy
-.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.-
8		5		59.000			33.600
8		4		68.750			43.250
8		3		66.333			21.333
8		2		43.500			40.000

# of hands	# of players	Royal Count-Strategy	Random-strategy
-.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.-
7		5		62.400			31.200
7		4		60.000			24.000
7		3		60.333			33.333
7		2		28.500			31.500

# of hands	# of players	Royal Count-Strategy	Random-strategy
-.--.--.--.--.--.--.--.--.--.--